In [ ]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from concurrent.futures import ThreadPoolExecutor, as_completed
import re

# Leemos nuestros dataframes
movies = pd.read_csv("./ml-latest-small/movies.csv", sep=",")
ratings = pd.read_csv("./ml-latest-small/ratings.csv", sep=",")
tags = pd.read_csv("./ml-latest-small/tags.csv", sep=",")
links = pd.read_csv("./ml-latest-small/links.csv", sep=",")

# Eliminamos nuelos
movies.dropna(inplace=True)
ratings.dropna(inplace=True)
tags.dropna(inplace=True)
links.dropna(inplace=True)

# Extraemos el año del título
movies['year'] = movies.title.str.extract("\((\d{4})\)", expand=True)
movies.year = pd.to_datetime(movies.year, format='%Y')
movies.year = movies.year.dt.year
movies.title = movies.title.str[:-7]
total=len(movies)
cuenta=0
nuestradatabase = pd.DataFrame()

def get_sinopsis(movie):
    titl = movies["title"]
    mov = movies.loc[titl==movie, "movieId"]
    movie_id = mov.iloc[0]
    movieId = links["movieId"]
    match = links.loc[movieId == movie_id, "tmdbId"]
    tmdb = match.iloc[0]
    tmdb = int(tmdb)
    tmdb = str(tmdb)
    moveiid = int(movie_id)
    moveiid = str(moveiid)
    url = "https://www.themoviedb.org/movie/" + tmdb
    #-----------------------
    chrome_options = Options()
    chrome_options.add_argument("--lang=en")
    driver = webdriver.Chrome(chrome_options=chrome_options)
    driver.get(url)
    html = driver.page_source
    soup = bs(html)
    driver.quit()
    #-----------------------
    sinop = soup.find_all("div", class_="overview")
    for p in sinop:
        sinopsis = p.get_text(strip=True)
    #-----------------------
    rating = soup.find_all("span", class_="icon")
    for per in rating:
        valor = per.get('class')[1]
    match = re.search(r'icon-r\d\d', valor)
    if match:
        # Obtenemos el valor de los dos números
        porcentaje = match.group()[6:]
    global nuestradatabase
    nuestradatabase = nuestradatabase.append({'moveId':moveiid,'sinopsis':sinopsis,'rating':porcentaje}, ignore_index=True)

with ThreadPoolExecutor() as executor:
    results = [executor.submit(get_sinopsis, movie) for movie in movies["title"]]
    for f in as_completed(results):
        pass
nuestradatabase.to_csv('sinopsisDB.csv', index=False)

In [2]:
nuestradatabase

,moveId,sinopsis,rating
0,17,Based on Jane Austen's classic novel of the Da...,75
1,19,"Summoned from an ashram in Tibet, Ace finds hi...",63
2,18,It's Ted the Bellhop's first night on the job....,57
3,16,"In early-1970s Las Vegas, Sam ""Ace"" Rothstein ...",80
4,1,"Led by Woody, Andy's toys live happily in his ...",80
...,...,...,...
8678,193579,Celebrating Jon Stewart's decade and a half ru...,77
8679,193587,A large scale catastrophe is occurring across ...,82
8680,193581,"Ciel learns of a ""Aurora Society"", that is rum...",77
8681,193585,A woman deals with the toxic water scandal in ...,70


In [22]:
pruebas = pd.read_csv("./ml-latest-small/movies_sinopsis.csv", sep=",")

In [3]:
import pandas as pd

In [27]:
pruebas

,movieId,title,genres,year,sinopsis,rating
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995.0,"Led by Woody, Andy's toys live happily in his ...",80
1,3,Grumpier Old Men,Comedy|Romance,1995.0,A family wedding reignites the ancient feud be...,64
2,4,Waiting to Exhale,Comedy|Drama|Romance,1995.0,"Cheated on, mistreated and stepped on, the wom...",63
3,5,Father of the Bride Part II,Comedy,1995.0,Just when George Banks has recovered from his ...,62
4,6,Heat,Action|Crime|Thriller,1995.0,Obsessive master thief Neil McCauley leads a t...,79
...,...,...,...,...,...,...
8456,193579,Jon Stewart Has Left the Building,Documentary,2015.0,Celebrating Jon Stewart's decade and a half ru...,77
8457,193581,Black Butler: Book of the Atlantic,Action|Animation|Comedy|Fantasy,2017.0,"Ciel learns of a ""Aurora Society"", that is rum...",77
8458,193585,Flint,Drama,2017.0,A woman deals with the toxic water scandal in ...,70
8459,193587,Bungo Stray Dogs: Dead Apple,Action|Animation,2018.0,A large scale catastrophe is occurring across ...,82


In [26]:
movies_sinopsis2=pruebas.drop(columns=["sinopsis","rating"])

In [28]:
movies_sinopsis2

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995.0
1,3,Grumpier Old Men,Comedy|Romance,1995.0
2,4,Waiting to Exhale,Comedy|Drama|Romance,1995.0
3,5,Father of the Bride Part II,Comedy,1995.0
4,6,Heat,Action|Crime|Thriller,1995.0
...,...,...,...,...
8456,193579,Jon Stewart Has Left the Building,Documentary,2015.0
8457,193581,Black Butler: Book of the Atlantic,Action|Animation|Comedy|Fantasy,2017.0
8458,193585,Flint,Drama,2017.0
8459,193587,Bungo Stray Dogs: Dead Apple,Action|Animation,2018.0


In [19]:
pruebas = pruebas.drop_duplicates()

In [21]:
pruebas.to_csv("./ml-latest-small/movies_sinopsis.csv", index=False)